<a href="https://colab.research.google.com/github/Lmalviya/machineTranslationTask/blob/main/Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scope of this notebook
1. understand Langchain (theory + implementation)
2. Understand LLaMa index
3. Build application

### Secrate keys

### Tutorial-01

In [3]:
!pip -q install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 18.2 MB/s eta 0:00:00


In [ ]:
!pip -q install langchain

## Tutorial-02

In the langchain there are three categories of chains
1. Generic Chains:
2. utility Chains:
3. Asynchronous Chains

- Generic Chains:
  - **LLM Chain**: this chain simply combines a prompt with an LLM and an output parser. the recommended way to do this is just to use LCEL.
  - **Transformation Chain**: This allows you to take something run some code in a function on the actual inputs or outputs of another chain
  - **Sequential Chain**:
- utility Chains:
  - **PAL chain**:
  - **SQL Database Chain**:
  - **Bash**:
  - **Requeests chain**:
  - **API chains**:



### Basic LLMChain = Fact Extraction

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain

from langchain.chains.conversation.memory import ConversationBufferMemory # store full conversation and give full conversation each time
from langchain.chains import ConversationChain

from langchain.chains.conversation.memory import ConversationSummaryMemory # store only summary of the conversation
from langchain.chains.conversation.memory import ConversationBufferWindowMemory  # only store latest conversation messagess
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory # store past conversation summary and also have latest conversation messages as it

from langchain.chains.conversation.memory import ConversationKGMemory # extract the information based on what said in the form of knowledge graph memory
#knowledge graph look like: I ('Sam', 'Human', 'is'), ('TV', 'true', 'is broken'), ('TV', 'weird sounds', 'makes'), ('TV', 'black', 'goes'), ('TV', 'black', 'goes'), ('TV', 'yes', 'is under warranty')

from langchain.chains.conversation.memory import ConversationEntityMemory # only store the entitys like {'Sam': "', 'TV': ''} and also some laters conversation as well



## Tutorial-03

In [4]:
import os
from langchain.llms import HuggingFaceHub
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

os. environ ["HUGGINGFACEHUB_API_TOKEN"] = huggingface_token

In [16]:
flan_t5 = HuggingFaceHub(
              repo_id="google/flan-t5-base",
              model_kwargs={ "temperature": 0 }
              )

In [19]:
conversation = ConversationChain(
    llm=flan_t5,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [20]:
conversation.predict(input="Hi, there! i am same.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, there! i am same.
AI:

> Finished chain.


'Human: Hi, there! I am the same.'

In [21]:
conversation.predict(input="can you help me in some customer support?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, there! i am same.
AI: Human: Hi, there! I am the same.
Human: can you help me in some customer support?
AI:

> Finished chain.


'Human: Sure, I can help you with customer support.'

In [22]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [23]:
def chat_to_llm(chat_llm, tokenizer):
  conversation_total_tokens = 0
  new_conversation = ConversationChain(llm=chat_llm, verbose=True, memory=ConversationBufferMemory())

  while True:
    message = input("Human: ")
    if message =='exit':
      print(f"{conversation_total_tokens} token used in this total conersation!")
      break
    else:
      formated_prompt = conversation.prompt.format(input=message, history=memory.buffer)
      num_tokens = len(tokenizer.tokenize(formated_prompt))
      conversation_total_tokens += num_tokens
      print(f"tokens sent {num_tokens}")
      response = new_conversation.predict(input=message)
      response_num_tokens = len(tokenizer.tokenize(response))
      print(f"LLM: {response}")


In [24]:
chat_to_llm(flan_t5, tokenizer)

Human: hi, I am sam
tokens sent 115


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi, I am sam
AI:

> Finished chain.
LLM: Hello, how are you?
Human: I am good! can you help me to understand the Harry power novel?
tokens sent 123


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi, I am sam
AI: Hello, how are you?
Human: I am good! can you help me to understand the Harry power novel?
AI:

> Finished chain.
LLM: Human: Sure, I'd li